In [0]:
import numpy as np

edgesRDD = sc.textFile('/FileStore/tables/comp4651-project/musae_crocodile_edges.csv', 8)
header = edgesRDD.first()
edgesRDD = (
  edgesRDD.filter(lambda line: line != header)
      .map(lambda line: tuple(map(int, line.split(','))))
)

def join_list(list1, list2):
  if list1 is None:
    return list2
  if list2 is None:
    return list1
  else:
    return list1+list2

nodesRDD = (
  edgesRDD.groupByKey().mapValues(list)
    .fullOuterJoin(edgesRDD.map(lambda kv: (kv[1], kv[0])).groupByKey().mapValues(list))
    .mapValues(lambda v: join_list(v[0], v[1]))
).cache()

In [0]:
# PART 2.1 - node distance estimation by landmark approach

allNodes = nodesRDD.sortByKey().map(lambda kv: kv[0]).collect()
num_nodes = len(allNodes)
nodeid_to_pos = sc.broadcast(dict(zip(allNodes, range(num_nodes))))
allNodes = sc.broadcast(np.array(allNodes))

landmarkNodes = nodesRDD.mapValues(len).sortBy(lambda kv: kv[1], ascending=False).map(lambda kv: kv[0]).take(100) # use top 100 nodes with highest degree as 
landmarkNodes = np.array(landmarkNodes)

In [0]:
def BFS(node_neighbors_dist):
  node = node_neighbors_dist[0]
  neighbors = node_neighbors_dist[1][0]
  dist = node_neighbors_dist[1][1]
  for n in neighbors:
    yield (n, dist+1)
  yield (node, dist)

In [0]:
dists = nodesRDD.map(lambda node: (node[0], np.where(landmarkNodes == node[0], 0, np.inf)))

for i in range(10):
  update = (
    nodesRDD
      .join(dists)
      .flatMap(BFS)
  )
  dists = update.reduceByKey(np.minimum)

In [0]:
def cartesian_to_dist_pairs(node_dists):
  node1 = node_dists[0][0]
  node2 = node_dists[1][0]
  if (node1 == node2):
    return (node1, (node2, 0))

  dists1 = node_dists[0][1]
  dists2 = node_dists[1][1]
  dist = np.min(dists1+dists2)
  return (node1, (node2, dist))

def seqOp(x, y):
  node_pos = nodeid_to_pos.value[y[0]]
  dist = y[1]
  x[node_pos] = dist
  return x

dist_to_landmarks = dists.cache()
alldists = (
  dist_to_landmarks
    .cartesian(dist_to_landmarks)
    .map(cartesian_to_dist_pairs)
    .aggregateByKey(np.zeros(num_nodes), seqOp, lambda x,y: x+y)
)
    

In [0]:
dist_estimated_by_landmarks = alldists.collect()

In [0]:
# PART 2.2 - Actual Node distance

dists = nodesRDD.map(lambda node: (node[0], np.where(allNodes.value == node[0], 0, np.inf)))

for i in range(10):
  update = (
    nodesRDD
      .join(dists)
      .flatMap(BFS)
  )
  dists = update.reduceByKey(np.minimum)

In [0]:
dist = dists.take(20)